# Agent-based Transportation Model

In [1]:
from mesa import Model
from mesa.time import RandomActivation
import requests

from dpd.modeling import OriginDestinationDataFrame, PersonAgent, Vehicle, Zones
from dpd.osrm import OSRM
from dpd.shapely import random_point_in_polygon

In [2]:
class TransportationModel(Model):
    def __init__(self):
        self.schedule = RandomActivation(self)

    def step(self):
        self.schedule.step()

model = TransportationModel()

This is what was done in peopleModel.py

In [3]:
zones = Zones.from_uscensus("35", "2017")
zones.calculate_centroids()
zones.head(1)

,Name,Production,Attraction,state,county,tract,STATEFP,COUNTYFP,TRACTCE,NAME_y,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,centroid
GEOID,,,,,,,,,,,,,,,,,,,
35047957500,"Census Tract 9575, San Miguel County, New Mexico",4117,None,35,047,957500,35,047,957500,9575,Census Tract 9575,G5020,S,9014844373,51188901,+35.5028648,-104.5780441,"POLYGON ((-105.54769 35.78157, -105.54724 35.7...",POINT (35.502 -104.570)


In [4]:
od = OriginDestinationDataFrame.from_lodes("nm", "2017").sample(10)
od.head(1)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3183: DtypeWarning: Columns (28,29,31,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
trct_w,trct_h,,,,,,,,,,,,,
35001004735,35001000502,350010047352000,350010005022015,1,0,0,1,1,0,0,0,1,0,20190825


This is what was done in routeModel.py

In [5]:
url_base = "http://localhost:5000"
osrm = OSRM(region="north-america/us/new-mexico", profile="car")
p = osrm.routed()
from time import sleep
sleep(2)

for index, row in od.iterrows():
    for i in range(row["S000"]):
        person = PersonAgent(model=model)
        model.schedule.add(person)
        person.home = random_point_in_polygon(zones.loc[index[1]].geometry)
        person.work = random_point_in_polygon(zones.loc[index[0]].geometry)
        url = url_base + '/route/v1/driving/' + str(person.home.xy[0][0]) + ',' + str(person.home.xy[1][0]) + ';' + str(person.work.xy[0][0]) + ',' + str(person.work.xy[1][0])
        r = requests.get(url)
        person.route_geometry = r.json()['routes'][0]['geometry']

p.kill()

This is what was done in simulation.py

In [6]:
for i in range(10):
    model.step()